## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2847469142501036352
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15473766345370052211
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7052715648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13888012816401726520
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4453487007954403861
physical_device_desc: "device: XLA_GPU device"
]


In [33]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

In [34]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('train : x=%s, y=%s' %(x_train.shape,y_train.shape))
print('test : x=%s, y=%s'  %(x_test.shape,y_test.shape))

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std

## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

train : x=(50000, 32, 32, 3), y=(50000, 1)
test : x=(10000, 32, 32, 3), y=(10000, 1)


In [35]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [45]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape = (32,32,3),activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))


#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) 
classifier.add(BatchNormalization())
classifier.add(Dropout(0.5))

classifier.add(Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
classifier.add(BatchNormalization())

classifier.add(Dropout(0.3))

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)

from tensorflow.keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop
##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

Epoch 1/100
500/500 [==============================] - 21s 42ms/step - loss: 2.1511 - accuracy: 0.3384 - val_accuracy: 0.4804 - val_loss: 1.6967
Epoch 2/100
500/500 [==============================] - 21s 42ms/step - loss: 1.7130 - accuracy: 0.4628 - val_accuracy: 0.5412 - val_loss: 1.4812
Epoch 3/100
500/500 [==============================] - 21s 42ms/step - loss: 1.5568 - accuracy: 0.5082 - val_accuracy: 0.5208 - val_loss: 1.5709
Epoch 4/100
500/500 [==============================] - 21s 42ms/step - loss: 1.4651 - accuracy: 0.5323 - val_accuracy: 0.5469 - val_loss: 1.4470
Epoch 5/100
500/500 [==============================] - 21s 42ms/step - loss: 1.4031 - accuracy: 0.5468 - val_accuracy: 0.5822 - val_loss: 1.3063
Epoch 6/100
500/500 [==============================] - 21s 42ms/step - loss: 1.3560 - accuracy: 0.5643 - val_accuracy: 0.6244 - val_loss: 1.1797
Epoch 7/100
500/500 [==============================] - 21s 42ms/step - loss: 1.3265 - accuracy: 0.5738 - val_accuracy: 0.6131 - va

500/500 [==============================] - 21s 42ms/step - loss: 1.1937 - accuracy: 0.6236 - val_accuracy: 0.6662 - val_loss: 1.0647
Epoch 23/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1919 - accuracy: 0.6230 - val_accuracy: 0.6491 - val_loss: 1.1149
Epoch 24/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1824 - accuracy: 0.6303 - val_accuracy: 0.6780 - val_loss: 1.0382
Epoch 25/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1882 - accuracy: 0.6269 - val_accuracy: 0.6529 - val_loss: 1.1134
Epoch 26/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1816 - accuracy: 0.6297 - val_accuracy: 0.6732 - val_loss: 1.0416
Epoch 27/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1820 - accuracy: 0.6308 - val_accuracy: 0.6706 - val_loss: 1.0644
Epoch 28/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1740 - accuracy: 0.6330 - val_accuracy: 0.6734 - val_loss

500/500 [==============================] - 22s 44ms/step - loss: 1.1304 - accuracy: 0.6512 - val_accuracy: 0.6960 - val_loss: 0.9865
Epoch 66/100
500/500 [==============================] - 22s 44ms/step - loss: 1.1225 - accuracy: 0.6531 - val_accuracy: 0.6927 - val_loss: 0.9961
Epoch 67/100
500/500 [==============================] - 21s 42ms/step - loss: 1.1226 - accuracy: 0.6541 - val_accuracy: 0.6850 - val_loss: 1.0143
Epoch 68/100
500/500 [==============================] - 21s 43ms/step - loss: 1.1253 - accuracy: 0.6532 - val_accuracy: 0.6737 - val_loss: 1.0704
Epoch 69/100
500/500 [==============================] - 22s 43ms/step - loss: 1.1198 - accuracy: 0.6554 - val_accuracy: 0.6910 - val_loss: 1.0055
Epoch 70/100
500/500 [==============================] - 21s 43ms/step - loss: 1.1189 - accuracy: 0.6588 - val_accuracy: 0.6880 - val_loss: 1.0142
Epoch 71/100
500/500 [==============================] - 21s 43ms/step - loss: 1.1148 - accuracy: 0.6561 - val_accuracy: 0.6674 - val_loss

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [48]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.0161148 , 0.00197082, 0.20620728, 0.3594891 , 0.07750414,
        0.02291854, 0.23213898, 0.00256828, 0.08011653, 0.00097164]],
      dtype=float32)